In [1]:
import pandas as pd
from arcgis.features import FeatureLayer

In [2]:
def get_fs_data_spatial(service_url):
    feature_layer = FeatureLayer(service_url)
    sdf = pd.DataFrame.spatial.from_layer(feature_layer)
    return sdf

In [3]:
# We have two sheets in an excel file, one for california and one for nevada
# We want to combine them into one dataframe
# We need to standardize a traffic count station feature class
traffic_data = "data/AADT_JS_Updated20240312.xlsx"
ca_data = pd.read_excel(traffic_data, sheet_name='CA')
nv_data = pd.read_excel(traffic_data, sheet_name='NV')
all_data = pd.concat([ca_data, nv_data], ignore_index=True)

In [4]:
nv_data.rename(columns={'Station ID': 'Station_Number'}, inplace=True)

In [5]:
#Melt california data with 'Description' as the id vars
# 
years = [2022] 
ca_data_melted = pd.melt(ca_data, id_vars=['County','DESCRIPTION'], value_vars=years, var_name='year_of_count', value_name='count_value')

nv_data_melted = pd.melt(nv_data, id_vars=['Station_Number'], value_vars=years, var_name='year_of_count', value_name='count_value')
#all_data_melted = pd.concat([ca_data_melted, nv_data_melted], ignore_index=True)


In [6]:
count_locations = get_fs_data_spatial('https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/10')

In [7]:
#drop a list of columns from count_locations
columns_to_drop = ['BACK_PEAK_HR', 'BACK_PEAK_HR','BACK_PEAK_MONTH', 'BACK_AADT',
                   'AHEAD_PEAK_HR', 'AHEAD_PEAK_MONTH', 'AHEAD_AADT',
    'year_of_count', 'count_value', 'OBJECTID', 'GlobalID', 'created_date', 
                   'created_user', 'last_edited_date', 'last_edited_user']
count_locations_unique = count_locations.drop(columns_to_drop, axis=1)
count_locations_unique = count_locations_unique.drop_duplicates(subset=['SHAPE'])


In [8]:
count_locations_unique.to_csv('data/count_locations_unique.csv', index=False)

In [9]:
ca_location_mapping = pd.read_csv('data/location_mapping.csv')
#merge ca_data_melted with ca_location_mapping on 'county and site_name'
ca_data_melted = pd.merge(ca_data_melted, ca_location_mapping, on=['County', 'DESCRIPTION'], how='left')

In [10]:
ca_data_joined = pd.merge(count_locations_unique, ca_data_melted, on=['site_name','COUNTY'], how='left')
ca_data_joined = ca_data_joined.dropna(subset=['ROUTE'])

In [11]:
nv_data_melted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Station_Number  35 non-null     int64 
 1   year_of_count   35 non-null     object
 2   count_value     35 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 972.0+ bytes


In [ ]:
count_locations_unique.info()

In [12]:
nv_data_melted['Station_Number']=nv_data_melted['Station_Number'].astype(str)

In [13]:
nv_data_joined = pd.merge(count_locations_unique, nv_data_melted, on=['Station_Number'], how='left')
nv_data_joined = nv_data_joined.dropna(subset=['count_value'])

In [ ]:
import arcpy

gdb_path = r"F:\GIS\PROJECTS\Transportation\Traffic_Counts\Traffic_Counts\Traffic_Counts.gdb"
feature_class_name = "ca_traffic_counts_1"
output_fc = f"{gdb_path}\\{feature_class_name}"

# Create the feature class
arcpy.CreateFeatureclass_management(gdb_path, feature_class_name, "POINT", spatial_reference=4326)

# Add fields
# Add all non-geometry fields dynamically
for column in ca_data_joined.columns:
    if column not in ["longitude", "latitude", "SHAPE"]:  # Exclude geometry fields
        field_type = "TEXT" if ca_data_joined[column].dtype == object else "DOUBLE"  # Assign data types
        arcpy.AddField_management(output_fc, column, field_type)

# Get field names for insertion (excluding geometry)
field_names = [field.name for field in arcpy.ListFields(output_fc) if field.name not in ["OBJECTID", "SHAPE"]]

# Insert data dynamically
with arcpy.da.InsertCursor(output_fc, ["SHAPE@"] + field_names) as cursor:
    for _, row in ca_data_joined.iterrows():
        
        values = [row['SHAPE']] + [row[col] for col in field_names]  # Collect field values
        cursor.insertRow(values)

print(f"Feature class created at {output_fc}")

In [ ]:
output_fc = f"{gdb_path}\\{feature_class_name}"

# Convert SDF to a Feature Class using ArcPy
ca_data_joined.spatial.to_featureclass(location=output_fc)

In [ ]:
feature_class_name = "nv_traffic_counts"
nv_data_joined.spatial.to_featureclass(location=f"{gdb_path}\\{feature_class_name}")

In [ ]:
import arcpy

gdb_path = r"F:\GIS\PROJECTS\Transportation\Traffic_Counts\Traffic_Counts\Traffic_Counts.gdb"
feature_class_name = "nv_traffic_counts"
output_fc = f"{gdb_path}\\{feature_class_name}"

# Create the feature class
arcpy.CreateFeatureclass_management(gdb_path, feature_class_name, "POINT", spatial_reference=4326)

# Add fields
# Add all non-geometry fields dynamically
for column in ca_data_joined.columns:
    if column not in ["longitude", "latitude", "SHAPE"]:  # Exclude geometry fields
        field_type = "TEXT" if ca_data_joined[column].dtype == object else "DOUBLE"  # Assign data types
        arcpy.AddField_management(output_fc, column, field_type)

# Get field names for insertion (excluding geometry)
field_names = [field.name for field in arcpy.ListFields(output_fc) if field.name not in ["OBJECTID", "SHAPE"]]

# Insert data dynamically
with arcpy.da.InsertCursor(output_fc, ["SHAPE@"] + field_names) as cursor:
    for _, row in ca_data_joined.iterrows():
        point = arcpy.Point(row.longitude, row.latitude)  # Create point geometry
        values = [point] + [row[col] for col in field_names]  # Collect field values
        cursor.insertRow(values)

print(f"Feature class created at {output_fc}")